## 使用盘古预报
### 1. 库的导入及一些基本设置

In [ ]:
import os
import xarray as xr
import numpy as np

filepath = "/autodl-fs/data" # ERA5数据文件的路径
model_path = "/autodl-fs/data/pangu" # 盘古的onnx模型文件所在路径
forecast_start_time = "2022-9-7 18:00" # 起报时间
savepath = "/root/autodl-tmp/result_pangu" # 结果保存路径
run_time_step = 32 # 运行的时间步长

### 2. 读取并处理数据，并保存处理出来的数据

In [ ]:
levels = np.array([1000, 925, 850, 700, 600, 500, 400, 300, 250, 200, 150, 100, 50])
ds1 = xr.open_dataset(f"{filepath}/9_surface.nc").sel(time=forecast_start_time)
ds2 = xr.open_dataset(f"{filepath}/9_high.nc").sel(time=forecast_start_time)
ds2 = ds2.sel(level=levels)

lon, lat, level = ds2.longitude.data, ds2.latitude.data, ds2.level.data
nlev = len(level)
data_surf = np.stack([ds1[v].data for v in "msl u10 v10 t2m".split()], 0).astype(np.float32)
data_plev = np.stack([ds2[v].data for v in "zqtuv"], 0).astype(np.float32)
print(f"{data_surf.shape=}, {data_plev.shape=}")

os.path.exists(savepath) or os.mkdir(savepath)
np.savez(f"{savepath}/pangu_input_data.npz", **dict(data_surf = data_surf, data_plev = data_plev))

### 3. 读取输入数据和加载盘古模型

In [ ]:
ds = np.load(f"{savepath}/pangu_input_data.npz")
input_data_surf = ds["data_surf"]
input_data_plev = ds["data_plev"]

In [ ]:
import onnx
import onnxruntime as ort

model = onnx.load(f"{model_path}/pangu_weather_6.onnx")

# Set the behavier of onnxruntime
options = ort.SessionOptions()
options.enable_cpu_mem_arena=False
options.enable_mem_pattern = False
options.enable_mem_reuse = False
# Increase the number for faster inference and more memory consumption
options.intra_op_num_threads = 1

# Set the behavier of cuda provider
cuda_provider_options = {'arena_extend_strategy':'kSameAsRequested',}

# Initialize onnxruntime session for Pangu-Weather Models
ort_session = ort.InferenceSession(f"{model_path}/pangu_weather_6.onnx", sess_options=options, providers=[('CUDAExecutionProvider', cuda_provider_options)])


### 4. 运行模型执行预测

In [ ]:
for i in range(run_time_step):
    output_plev, output_surface = ort_session.run(None, {'input':input_data_plev, 'input_surface':input_data_surf})
    input_data_plev, input_data_surf = output_plev, output_surface
    # save the results
    np.savez(f"{savepath}/output{i + 1:02d}.npz", **dict(plev=output_plev, surf=output_surface))

### 5. 对预测数据做后处理转换成nc

In [ ]:
import pandas as pd

time = pd.date_range(forecast_start_time, periods = run_time_step + 1, freq='6h')
start_forecast_time = time[0].strftime("%Y%m%d%H")

for i in range(run_time_step):
    file = f"{savepath}/output{i + 1:02d}.npz"
    ds = np.load(file)
    data_surf = ds["surf"]
    data_plev = ds["plev"]
    if i == 0:
        nv, nlev, ny, nx = data_plev.shape
        data1 = np.zeros((nv, run_time_step, nlev, ny, nx), dtype=np.float32)
        data2 = np.zeros((4, run_time_step, ny, nx), dtype=np.float32)
    data1[:, i] = data_plev
    data2[:, i] = data_surf
    os.remove(file)

data_vars = {v:(("time", "lat", "lon"), data2[i]) for i, v in enumerate("msl u10 v10 t2m".split())}
data_vars.update({v:(("time", "level", "lat", "lon"), data1[i]) for i, v in enumerate("zqtuv")})
ds = xr.Dataset(data_vars=data_vars, coords=dict(time=(("time", ), time[1:]), level=(("level", ), level), 
                                                 lat=(("lat", ), lat), lon=(("lon", ), lon)))
ds.to_netcdf(f"{savepath}/pangu{start_forecast_time}_lead{run_time_step:02d}.nc")

In [ ]:
!ls -lsh /root/autodl-tmp/result_pangu